In [ ]:
def loguniform(low=0, high=1):
    val = np.exp(np.random.uniform(0, 1, None))
    scaled_val = (((val - np.exp(0)) * (high - low)) / (np.exp(1) - np.exp(0))) + low
    return scaled_val

In [ ]:
best_auc=0
best_param={0,0,0}
all_auc=[]
all_param=[]
best_aucpr=0
all_aucpr=[]
best_parampr={0,0,0}
time=[]
confusion=[]

start=datetime.now()

for i in range(4,5):
    for k in range(12,13):

        print('run ' + str(i) + '.' + str(k) + ' started at ' + str(datetime.now()))
        np.random.seed(i)
        train_df_split = train_df_ready.reindex(np.random.permutation(train_df_ready.index))
        val_ids = list(train_df_split['EC batch'].unique())[-int(val_ratio*len(train_df_split['EC batch'].unique())):]
        val_df_pre = train_df_ready[train_df_ready['EC batch'].isin(val_ids)]
        train_df_pre = train_df_ready[~train_df_ready['EC batch'].isin(val_ids)]

        n_estimators = np.random.randint(150, 1000)
        max_features = loguniform(0.01, 0.9)
        max_depth = np.random.randint(3, 9)

        X_train = train_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_train = train_df_pre['Adherence']
        X_val = val_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_val = val_df_pre['Adherence']

        rfc_model= RandomForestClassifier(n_estimators=n_estimators,max_features=max_features,max_depth = max_depth,class_weight='balanced')

#         print(datetime.now())
        rfc_model.fit(X_train, y_train)
#         print(datetime.now())
        ns_probs = [0 for _ in range(len(y_val))]

        lr_probs = rfc_model.predict_proba(X_val)
        # keep probabilities for the positive outcome only
        lr_probs = lr_probs[:, 1]

        # calculate the precision-recall auc
        precision, recall, _ = precision_recall_curve(y_val, lr_probs)
        pr_auc = auc(recall, precision)
#         print(pr_auc)

        # calculate scores
        ns_auc = roc_auc_score(y_val, ns_probs)
        lr_auc = roc_auc_score(y_val, lr_probs)
        # summarize scores
#         print(lr_auc)
        predictions = rfc_model.predict(X_val)
#         print(classification_report(y_val,predictions))

        confusion.append(precision_recall_fscore_support(y_val, predictions, average=None))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='macro'))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='weighted'))

        if lr_auc>best_auc:
            best_auc=lr_auc
            best_param={'run ' +str(i)+'.'+str(k),n_estimators,max_features,max_depth}

        if pr_auc>best_aucpr:
            best_aucpr=pr_auc
            best_parampr={'run ' +str(i)+'.'+str(k),n_estimators,max_features,max_depth}

        all_auc.append(lr_auc)
        all_aucpr.append(pr_auc)
        all_param.append(['n_estimators= ' +str(n_estimators),'max_features= ' +str(max_features),
                          'max_depth= ' +str(max_depth)])



        print('run ' + str(i) + '.' + str(k) + ' ended at ' + str(datetime.now()))
        
print('cv ended at ' + str(datetime.now()))

end=datetime.now()
time.append(end-start)

# # fileexport_confusion='Data/confusion_rf.csv'
# fileexport_time='Data/time_rf.csv'
# fileexport_bestauc='Data/best_auc_rf.csv'
# fileexport_bestparam='Data/best_param_rf.csv'
# fileexport_bestaucpr='Data/best_aucpr_rf.csv'
# fileexport_bestparampr='Data/best_parampr_rf.csv'
# fileexport_auc='Data/all_auc_rf.csv'
# fileexport_aucpr='Data/all_aucpr_rf.csv'
# fileexport_param='Data/all_param_rf.csv'
# # pd.DataFrame(best_auc).to_csv(fileexport_bestauc,index=False, header=False)
# pd.DataFrame(best_param).to_csv(fileexport_bestparam,index=False, header=False)
# # pd.DataFrame(best_aucpr).to_csv(fileexport_bestaucpr,index=False, header=False)
# pd.DataFrame(best_parampr).to_csv(fileexport_bestparampr,index=False, header=False)
# pd.DataFrame(all_auc).to_csv(fileexport_auc,index=False, header=False)
# pd.DataFrame(all_aucpr).to_csv(fileexport_aucpr,index=False, header=False)
# pd.DataFrame(all_param).to_csv(fileexport_param,index=False, header=False)
# pd.DataFrame(time).to_csv(fileexport_time,index=False, header=False)
# # pd.DataFrame(confusion).to_csv(fileexport_confusion, index=False, header=False)



 